<a href="https://colab.research.google.com/github/yugeswari-33/YUGESWARI-0309/blob/main/kmeans_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans as SKKMeans
import os
import csv
import json

# For reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Output directories (Colab default is /content)
OUT_DIR = "/content/kmeans_outputs"
os.makedirs(OUT_DIR, exist_ok=True)
print("Output directory:", OUT_DIR)# Cell 2: Generate synthetic dataset
# As requested: 500 samples, 4 centers, std deviation 1.5
n_samples = 500
n_centers = 4
cluster_std = 1.5
random_state = RANDOM_SEED

X, y_true = make_blobs(n_samples=n_samples,
                       centers=n_centers,
                       cluster_std=cluster_std,
                       random_state=random_state)

# Save dataset to disk for later reuse
np.save(os.path.join(OUT_DIR, "X.npy"), X)
np.save(os.path.join(OUT_DIR, "y_true.npy"), y_true)

print("Generated X shape:", X.shape)
print("Saved dataset as X.npy and y_true.npy in", OUT_DIR)# Cell 3: K-Means implementation from scratch
# Functions: kmeans_pp_init, random_init, assign_clusters, compute_centroids, compute_inertia, kmeans_scratch

def squared_euclidean(a, b):
    # a: (n_samples, n_features)
    # b: (n_centroids, n_features)
    # returns distances shape (n_samples, n_centroids)
    return np.sum((a[:, None, :] - b[None, :, :]) ** 2, axis=2)

def random_init(X, K, random_state=None):
    rng = np.random.RandomState(random_state)
    indices = rng.choice(X.shape[0], size=K, replace=False)
    return X[indices].copy()

def kmeans_pp_init(X, K, random_state=None):
    rng = np.random.RandomState(random_state)
    n, d = X.shape
    centers = np.zeros((K, d), dtype=X.dtype)
    # 1) choose first center uniformly
    first_idx = rng.randint(0, n)
    centers[0] = X[first_idx]
    # 2) choose remaining
    for k in range(1, K):
        dist2 = np.min(squared_euclidean(X, centers[:k]), axis=1)  # dist^2 to nearest existing center
        probs = dist2 / np.sum(dist2)
        # Choose next index by weighted probability
        next_idx = rng.choice(n, p=probs)
        centers[k] = X[next_idx]
    return centers

def assign_clusters(X, centers):
    # returns labels (n_samples,) and distances to assigned center (n_samples,)
    dists = squared_euclidean(X, centers)
    labels = np.argmin(dists, axis=1)
    assigned_dists = dists[np.arange(X.shape[0]), labels]
    return labels, assigned_dists

def compute_centroids(X, labels, K):
    d = X.shape[1]
    centers = np.zeros((K, d), dtype=X.dtype)
    for k in range(K):
        members = X[labels == k]
        if len(members) == 0:
            # If a cluster lost all members, reinitialize randomly (common strategy)
            centers[k] = X[np.random.randint(0, X.shape[0])]
        else:
            centers[k] = members.mean(axis=0)
    return centers

def compute_inertia(assigned_dists):
    # inertia is sum of squared distances to assigned center
    return np.sum(assigned_dists)

def kmeans_scratch(X, K=4, init="kmeans++", max_iters=300, tol=1e-6, random_state=None, verbose=False):
    """
    X: data array (n_samples, n_features)
    K: number of clusters
    init: "random" or "kmeans++"
    returns a dict with:
      centers (K, d), labels (n,), inertia_history list, n_iter, converged boolean
    """
    if init == "random":
        centers = random_init(X, K, random_state)
    elif init == "kmeans++":
        centers = kmeans_pp_init(X, K, random_state)
    else:
        raise ValueError("init must be 'random' or 'kmeans++'")

    inertia_history = []
    prev_inertia = None
    for it in range(1, max_iters + 1):
        labels, assigned_dists = assign_clusters(X, centers)
        inertia = compute_inertia(assigned_dists)
        inertia_history.append(inertia)
        if verbose:
            print(f"Iter {it}, inertia {inertia:.6f}")
        # update
        new_centers = compute_centroids(X, labels, K)
        # check centroid shifts
        center_shifts = np.sqrt(np.sum((centers - new_centers) ** 2, axis=1))
        centers = new_centers
        # convergence criteria: small change in inertia or centers
        if prev_inertia is not None and abs(prev_inertia - inertia) < tol:
            if verbose:
                print(f"Converged by inertia change at iter {it}")
            return {"centers": centers, "labels": labels, "inertia_history": inertia_history, "n_iter": it, "converged": True}
        if np.all(center_shifts < tol):
            if verbose:
                print(f"Converged by center shifts at iter {it}")
            return {"centers": centers, "labels": labels, "inertia_history": inertia_history, "n_iter": it, "converged": True}
        prev_inertia = inertia
    # max iters reached
    return {"centers": centers, "labels": labels, "inertia_history": inertia_history, "n_iter": max_iters, "converged": False}# Cell 4: Run the scratch KMeans with K=4 using both initialization strategies
K = 4
max_iters = 300
tol = 1e-6

# Run with k-means++ init
res_pp = kmeans_scratch(X, K=K, init="kmeans++", max_iters=max_iters, tol=tol, random_state=RANDOM_SEED, verbose=True)

# Run with random init (same seed for reproducibility)
res_rand = kmeans_scratch(X, K=K, init="random", max_iters=max_iters, tol=tol, random_state=RANDOM_SEED, verbose=True)

# Save numeric outputs
np.save(os.path.join(OUT_DIR, "centers_kpp.npy"), res_pp["centers"])
np.save(os.path.join(OUT_DIR, "labels_kpp.npy"), res_pp["labels"])
np.save(os.path.join(OUT_DIR, "inertia_history_kpp.npy"), np.array(res_pp["inertia_history"]))

np.save(os.path.join(OUT_DIR, "centers_rand.npy"), res_rand["centers"])
np.save(os.path.join(OUT_DIR, "labels_rand.npy"), res_rand["labels"])
np.save(os.path.join(OUT_DIR, "inertia_history_rand.npy"), np.array(res_rand["inertia_history"]))

# Also save a JSON summary
summary = {
    "kpp": {"n_iter": res_pp["n_iter"], "final_inertia": float(res_pp["inertia_history"][-1]), "converged": bool(res_pp["converged"])},
    "random": {"n_iter": res_rand["n_iter"], "final_inertia": float(res_rand["inertia_history"][-1]), "converged": bool(res_rand["converged"])}
}
with open(os.path.join(OUT_DIR, "summary.json"), "w") as f:
    json.dump(summary, f, indent=2)

print("Saved scratch KMeans outputs and summary.")
print("Summary:", summary)# Cell 5: Run scikit-learn KMeans for comparison
sk_kmeans = SKKMeans(n_clusters=K, init="k-means++", n_init=10, random_state=RANDOM_SEED)
sk_kmeans.fit(X)
sk_centers = sk_kmeans.cluster_centers_
sk_labels = sk_kmeans.labels_
sk_inertia = sk_kmeans.inertia_

# Save sklearn outputs
np.save(os.path.join(OUT_DIR, "centers_sklearn.npy"), sk_centers)
np.save(os.path.join(OUT_DIR, "labels_sklearn.npy"), sk_labels)
with open(os.path.join(OUT_DIR, "sklearn_inertia.txt"), "w") as f:
    f.write(str(sk_inertia))

print("Sklearn final inertia:", sk_inertia)
print("Sklearn centers:\n", sk_centers)# Cell 6: Plot final clusters for scratch KMeans (kmeans++)
def plot_clusters(X, labels, centers, title, fname):
    plt.figure(figsize=(8,6))
    # scatter by label
    for k in np.unique(labels):
        pts = X[labels == k]
        plt.scatter(pts[:,0], pts[:,1], s=20, label=f"Cluster {k}", alpha=0.6)
    plt.scatter(centers[:,0], centers[:,1], s=200, marker='X', edgecolor='k', linewidth=1.2, label='Centers')
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(fname)
    plt.close()
    print("Saved plot:", fname)

plot_clusters(X, res_pp["labels"], res_pp["centers"], "Scratch KMeans (kmeans++) - Final Clusters", os.path.join(OUT_DIR, "clusters_scratch_kpp.png"))
plot_clusters(X, res_rand["labels"], res_rand["centers"], "Scratch KMeans (random init) - Final Clusters", os.path.join(OUT_DIR, "clusters_scratch_rand.png"))
plot_clusters(X, sk_labels, sk_centers, "Scikit-learn KMeans - Final Clusters", os.path.join(OUT_DIR, "clusters_sklearn.png"))# Cell 7: Plot inertia vs iterations
plt.figure(figsize=(8,6))
plt.plot(res_pp["inertia_history"], label="Scratch (kmeans++)")
plt.plot(res_rand["inertia_history"], label="Scratch (random)")
# Mark final sklearn inertia as horizontal line
plt.hlines(sk_inertia, 0, max(len(res_pp["inertia_history"]), len(res_rand["inertia_history"])),
           linestyles='dashed', label=f"sklearn final inertia = {sk_inertia:.2f}")
plt.xlabel("Iteration")
plt.ylabel("Inertia (sum squared distances)")
plt.title("Inertia vs Iteration")
plt.legend()
plt.tight_layout()
inertia_plot_path = os.path.join(OUT_DIR, "inertia_vs_iter.png")
plt.savefig(inertia_plot_path)
plt.close()
print("Saved inertia plot:", inertia_plot_path)

# Save inertia histories to CSV for inspection
import pandas as pd
df_inertia = pd.DataFrame({
    "iter": np.arange(1, len(res_pp["inertia_history"]) + 1),
    "inertia_kpp": res_pp["inertia_history"],
})
# If random has fewer or same length, align
df_inertia["inertia_rand"] = pd.Series(res_rand["inertia_history"]).reindex(df_inertia.index).values
df_inertia.to_csv(os.path.join(OUT_DIR, "inertia_histories.csv"), index=False)
print("Saved inertia_histories.csv")# Cell 8: Generate a text-based report and save to file
report_lines = []
report_lines.append("K-Means Project Report")
report_lines.append("======================")
report_lines.append(f"Dataset: {n_samples} samples, K={K}, std={cluster_std}")
report_lines.append("")
report_lines.append("Scratch K-Means (kmeans++)")
report_lines.append(f" - Converged: {res_pp['converged']}")
report_lines.append(f" - Iterations: {res_pp['n_iter']}")
report_lines.append(f" - Final inertia: {res_pp['inertia_history'][-1]:.6f}")
report_lines.append(f" - Centroids:\n{res_pp['centers']}")
report_lines.append("")
report_lines.append("Scratch K-Means (random init)")
report_lines.append(f" - Converged: {res_rand['converged']}")
report_lines.append(f" - Iterations: {res_rand['n_iter']}")
report_lines.append(f" - Final inertia: {res_rand['inertia_history'][-1]:.6f}")
report_lines.append(f" - Centroids:\n{res_rand['centers']}")
report_lines.append("")
report_lines.append("Scikit-learn KMeans")
report_lines.append(f" - Final inertia: {sk_inertia:.6f}")
report_lines.append(f" - Centroids:\n{sk_centers}")
report_lines.append("")
report_lines.append("Interpretation notes:")
report_lines.append("- Compare the final inertia numbers: lower inertia indicates closer-fitting clusters (but not always 'better' semantically).")
report_lines.append("- Differences between scratch and sklearn can come from initialization differences and how empty clusters are handled.")
report_lines.append("- Centroids represent cluster centers (mean of assigned points). You can interpret clusters by looking at centroid coordinates and nearby points.")
report_text = "\n".join(report_lines)

report_path = os.path.join(OUT_DIR, "report.txt")
with open(report_path, "w") as f:
    f.write(report_text)

print("Saved report:", report_path)
print("\n--- Report Preview ---\n")
print(report_text[:1000])  # print first 1000 chars of report for quick preview

Output directory: /content/kmeans_outputs
Generated X shape: (500, 2)
Saved dataset as X.npy and y_true.npy in /content/kmeans_outputs
Iter 1, inertia 9043.439728
Iter 2, inertia 4331.955262
Iter 3, inertia 4293.702737
Iter 4, inertia 4293.004556
Converged by center shifts at iter 4
Iter 1, inertia 39028.753653
Iter 2, inertia 9121.428233
Iter 3, inertia 4957.432464
Iter 4, inertia 2440.541328
Iter 5, inertia 2133.242998
Iter 6, inertia 2126.231043
Iter 7, inertia 2126.039402
Converged by center shifts at iter 7
Saved scratch KMeans outputs and summary.
Summary: {'kpp': {'n_iter': 4, 'final_inertia': 4293.004556228741, 'converged': True}, 'random': {'n_iter': 7, 'final_inertia': 2126.0394018716693, 'converged': True}}
Sklearn final inertia: 2126.0394018716693
Sklearn centers:
 [[ 4.76279743  1.89919856]
 [-8.64343131  7.55417397]
 [-7.06163376 -6.91663151]
 [-2.65633217  9.08649366]]
Saved plot: /content/kmeans_outputs/clusters_scratch_kpp.png
Saved plot: /content/kmeans_outputs/cluste